In [ ]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread, imsave
from skimage.filters import gaussian, threshold_otsu
from skimage.morphology import erosion, dilation, square
from skimage.measure import label, regionprops_table
from skimage.segmentation import clear_border
from skimage.draw import circle_perimeter
from skimage.color import
from tqdm import tqdm

In [ ]:
# Parameter settings
input_dir = r"path/to/your/input_images"
output_dir = r"path/to/your/output_folder"
gaussian_sigma = 1
dil = 1
ero = 5
area_min = 5000
area_max = 20000
intensity_thresh = 2000
circularity_thresh = 0.9
image_export = False

def filter_by_area_and_circularity(labeled_img, area_min, area_max, circularity_thresh):
    from skimage.measure import regionprops
    out_img = np.zeros_like(labeled_img)
    for region in regionprops(labeled_img):
        area = region.area
        perimeter = region.perimeter if region.perimeter != 0 else 1
        circularity = 4 * np.pi * area / (perimeter ** 2)
        if area_min <= area <= area_max and circularity >= circularity_thresh:
            out_img[labeled_img == region.label] = region.label
    return out_img

In [ ]:
# Get all d1 files
d1_files = sorted([f for f in os.listdir(input_dir) if f.endswith("d1.TIF")])

results = []

for fname in tqdm(d1_files, desc="Processing"):
    img_path = os.path.join(input_dir, fname)
    img = imread(img_path, plugin='pil')  # Read image while preserving 16-bit depth

    # If the image is 3D and the first dimension is channels (2), take the first channel
    if img.ndim == 3 and img.shape[0] == 2:
        img = img[0]
    elif img.ndim == 3 and img.shape[2] == 2:
        img = img[:, :, 0]

    # Preprocessing + binarization
    img_blur = gaussian(img, sigma=gaussian_sigma)
    thresh = threshold_otsu(img_blur)
    binary = img_blur > thresh
    binary = erosion(dilation(binary, square(dil)), square(ero))
    binary = clear_border(binary)
    binary = binary.astype(np.uint8)

    # Label and filter circles
    labeled = label(binary)
    # Filtering is performed by the helper function defined above (or an equivalent implementation)
    labeled_filtered = filter_by_area_and_circularity(
        labeled, area_min, area_max, circularity_thresh
    )

    # Get region properties
    props = regionprops_table(labeled_filtered, properties=["label", "area", "centroid"])
    df = pd.DataFrame(props)
   # Compute diameter in pixels
    df["diameter"] = np.sqrt(4 * df["area"] / np.pi)

    # Convert to real units (µm)
    pixel_size_um = 275 / 445  # Micrometers per pixel ≈ 0.61798
    df["diameter_real(um)"] = df["diameter"] * pixel_size_um

    # Add filename information
    df["filename"] = fname
    results.append(df)

    # Optional image export: draw circle perimeters on the 16-bit grayscale image using maximum intensity
    if image_export:
        out_img = img.copy()
        for i in range(len(df)):
            cy, cx = int(df["centroid-0"][i]), int(df["centroid-1"][i])
            radius = int(df["diameter"][i] / 2)
            rr, cc = circle_perimeter(cy, cx, radius)
            rr, cc = np.clip(rr, 0, img.shape[0] - 1), np.clip(cc, 0, img.shape[1] - 1)
            out_img[rr, cc] = 65535  # 16-bit maximum value for highlighting

        outname = os.path.join(output_dir, fname.replace(".TIF", "_labeled.TIF"))
        imsave(outname, out_img)

# Concatenate and save the result table
all_df = pd.concat(results, ignore_index=True)
all_df.to_csv(os.path.join(output_dir, "circle_analysis_result.csv"), index=False)

Processing: 100%|██████████| 224/224 [00:51<00:00,  4.33it/s]
